In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams["figure.figsize"] = [12,12]
#If you have problems with latex at matplotlib just comment next two lines, this might help
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [6]:
a = [[0.01,0.05],[0.01,0.1]]
b = [0.05,0.1]
c = 0.01
N = 2
def func(x):
    return np.matmul(np.matmul(np.transpose(x), a), x) + np.matmul(np.transpose(b), x) + c
def f_grad(x):
    ans = []
    for i in range(N):
        t = 0
        for j in range(N):
            t += a[i][j] * x[j]
        t += a[i][i] * x[i] + b[i]
        ans.append(t)
    return np.array(ans)
def p(x):
    new_x = []
    for i in range(N):
        if x[i] > 1:
            new_x.append(1)
        else:
            if x[i] < 0:
                new_x.append(0)
            else:
                new_x.append(x[i])
    return np.array(new_x)
            

def f_grad_2():
    ans = [[]]
    for i in range(N):
        for j in range(N):
            if not i == j:
                ans[i].append(a[i][j])
            else:
                ans[i].append(2 * a[i][i])
        if not i == N - 1:
            ans.append([])
    return np.array(ans)
def eigenvalues():
    w, v = np.linalg.eig(f_grad_2())
    return w

In [7]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
def animate_trajectory(traj):
    fig, ax = plt.subplots()
    n = len(traj)
    def step(t):
        ax.cla()
        ax.plot([0], [0], 'o', color='green')
        #Level contours
        delta = 0.025
        x = np.arange(-3, 3, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        #print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [0.05, 0.15, 0.3], colors=['blue', 'purple', 'red'])

        ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], color='black')
        ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], 'o', color='black')
        
        fix_scaling(ax)
        ax.axis('off')

    return FuncAnimation(fig, step,
                     frames=range(n), interval=600)

In [8]:
lambdas = list(eigenvalues())
alpha = 1 / (lambdas[0] + 1)
beta = (np.sqrt(lambdas[0]) - np.sqrt(lambdas[N - 1])) / (np.sqrt(lambdas[0]) + np.sqrt(lambdas[N - 1]))
traj = []
x_start = np.array([1, 1])
traj.append(x_start.copy())
cur_x = x_start.copy()
cur_y = x_start.copy()

for i in range(15):
    t = cur_x
    cur_x = p(cur_y - alpha * f_grad(cur_y))
    cur_y = cur_x + beta * (cur_x - t)
    traj.append(cur_x.copy())
    
#print(traj)
base_animation = animate_trajectory(traj)
HTML(base_animation.to_html5_video())    